Курс валют постоянно находиться в нестабильном состоянии. Отслеживать его вручную не так удобно, поэтому предлагаем создать программу на Питон для автоматического трекинга.

Наша программа будет работать по следующему алгоритму:

1. Парсим (копируем) данные из страницы поисковой выдачи Google;

2. Проверяем изменения курса с момента запуска программы;

3. Отправляем письмо на электронную почту при сильном изменении курса.



Первое с чем стоит определиться, так это откуда брать курсы валют. Честно говоря, то не вижу смысла платить за какой-либо API, когда можно скопировать спарсить данные прямиком с поисковой выдачи Google или если хотите с других сайтов, которые предоставляют схожий функционал. Таких данных нам вполне хватит, ведь всё равно вряд ли какой-либо API будет предоставлять ещё более точную информацию в отличии от Google.

Первое что необходимо сделать - это установить всё необходимое. Нам потребуется установить **Python** с официального сайта, скачать **IDE**, можно использовать **PyCharm** и далее создать внутри программы новый проект.



Внутри созданного проекта создаем новый файл с названием **main.py**. После этого сразу же через терминал выполняем установку библиотек: **requests**, а также **beautiful soup**

In [ ]:
!pip install requests bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=8a2088ad7d3fc24db45106c403f914b9caa3fb297ee24015e9abbfec201cea09
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


Библиотека **requests** позволяет обращаться к необходимому сайту и копировать всю его **HTML** разметку. Внутри **HTML** разметки выбрать нужные данные очень сложно, если использовать стандартные функции **Python** по типу: **replace**, **join** и прочих. Поэтому мы используем вторую библиотеку **beautiful soup** для быстрой и комфортной выборки необходимых данных из **HTML**.



Далее нам необходимо прописать весь код для корректной работы нашей программы. Весь код показан ниже:

In [ ]:
import requests # Модуль для обработки URL
from bs4 import BeautifulSoup # Модуль для работы с HTML
import time # Модуль для остановки программы
import smtplib # Модуль для работы с почтой

# Основной класс
class Currency:
	# Ссылка на нужную страницу
	DOLLAR_RUB = 'https://www.google.com/search?sxsrf=ALeKk01NWm6viYijAo3HXYOEQUyDEDtFEw%3A1584716087546&source=hp&ei=N9l0XtDXHs716QTcuaXoAg&q=%D0%B4%D0%BE%D0%BB%D0%BB%D0%B0%D1%80+%D0%BA+%D1%80%D1%83%D0%B1%D0%BB%D1%8E&oq=%D0%B4%D0%BE%D0%BB%D0%BB%D0%B0%D1%80+&gs_l=psy-ab.3.0.35i39i70i258j0i131l4j0j0i131l4.3044.4178..5294...1.0..0.83.544.7......0....1..gws-wiz.......35i39.5QL6Ev1Kfk4'
	# Заголовки для передачи вместе с URL
	headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}

	current_converted_price = 0
	difference = 5 # Разница после которой будет отправлено сообщение на почту

	def __init__(self):
		# Установка курса валюты при создании объекта
		self.current_converted_price = float(self.get_currency_price().replace(",", "."))

	# Метод для получения курса валюты
	def get_currency_price(self):
		# Парсим всю страницу
		full_page = requests.get(self.DOLLAR_RUB, headers=self.headers)

		# Разбираем через BeautifulSoup
		soup = BeautifulSoup(full_page.content, 'html.parser')

		# Получаем нужное для нас значение и возвращаем его
		convert = soup.findAll("span", {"class": "DFlfde", "class": "SwHCTb", "data-precision": 2})
		return convert[0].text

	# Проверка изменения валюты
	def check_currency(self):
		currency = float(self.get_currency_price().replace(",", "."))
		if currency >= self.current_converted_price + self.difference:
			print("Курс сильно вырос, может пора что-то делать?")
			self.send_mail()
		elif currency <= self.current_converted_price - self.difference:
			print("Курс сильно упал, может пора что-то делать?")
			self.send_mail()

		print("Сейчас курс: 1 доллар = " + str(currency))
		time.sleep(3) # Засыпание программы на 3 секунды
		self.check_currency()

	# Отправка почты через SMTP
	def send_mail(self):
		server = smtplib.SMTP('smtp.gmail.com', 587)
		server.ehlo()
		server.starttls()
		server.ehlo()

		server.login('secretmillionera@gmail.com', 'Secretmillionera1968')

		subject = 'Currency mail'
		body = 'Currency has been changed!'
		message = f'Subject: {subject}\n{body}'

		server.sendmail(
			'admin@itproger.com',
			'secretmillionera@gmail.com',
			message
		)
		server.quit()

# Создание объекта и вызов метода
currency = Currency()
currency.check_currency()

Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас курс: 1 доллар = 77.36
Сейчас кур

KeyboardInterrupt: ignored